In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path='drive/My Drive/CNN without cycling/Images/Subject3_Figures_50rps/'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sys,os
import random
from sklearn.model_selection import train_test_split

In [0]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 16
# percentage of training set to use as validation
test_size = 0.2
valid_size = 0.2
train_size = 0.8

# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

dataset = datasets.ImageFolder(path, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, shuffle = True)

# specify the image classes
classes = ['L1_Standing_still','L2_Sitting_and_relaxing', 'L3_Lying_down', 'L4_Walking', 'L5_Climbing_stairs','L9_Cycling', 'L10_Jogging', 'L11_Running']

# obtain training indices that will be used for validation
num_data = len(dataset)
indices = list(range(num_data))
np.random.seed(3)
np.random.shuffle(indices)
split = int(np.floor(test_size * num_data))
train_idx1, test_idx = indices[split:], indices[:split]

num_train_data = len(dataset) - split
split2 = int(np.floor(valid_size * num_train_data))
train_idx, valid_idx = train_idx1[split2:], train_idx1[:split2]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
print(len(train_idx))
print(len(test_idx))
print(len(valid_idx))

615
192
153


In [0]:
import torch.nn as nn
import torch.nn.functional as F
#35
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 144x32x4 image tensor)
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1)

        # convolutional layer (sees 72x16x16 tensor)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        
        # convolutional layer (sees 36x8x32 tensor)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 18x8x32 tensor)
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.pool1 = nn.MaxPool2d(3, 3)
        # linear layer (64 * 9 * 9 -> 500)
        self.fc1 = nn.Linear(64 * 9 * 9, 3500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(3500, 1500)
       # self.fc3 = nn.Linear(1500,700)
       # self.fc4 = nn.Linear(700, 300)
        self.fc3 = nn.Linear(1500, 700)
        self.fc4 = nn.Linear(700, 8)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        # flatten image input
        x = x.view(-1, 64 * 9 * 9)
        # add dropout layer
        #x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
       # x = F.relu(self.fc4(x))
       # x = self.dropout(x)
        x = self.fc4(x)
        return x
# create a complete CNN
model = Net()
print(model)
# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

Net(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=5184, out_features=3500, bias=True)
  (fc2): Linear(in_features=3500, out_features=1500, bias=True)
  (fc3): Linear(in_features=1500, out_features=700, bias=True)
  (fc4): Linear(in_features=700, out_features=8, bias=True)
  (dropout): Dropout(p=0.2)
)


In [0]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [0]:
# number of epochs to train the model
n_epochs = 1000

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
       # torch.save(model.state_dict(), 'model_cifar.pt')
        torch.save(model.state_dict(), 'drive/My Drive/CNN without cycling/model_augmented2.pt')
        valid_loss_min = valid_loss


Epoch: 1 	Training Loss: 1.332583 	Validation Loss: 0.331547
Validation loss decreased (inf --> 0.331547).  Saving model ...
Epoch: 2 	Training Loss: 1.332160 	Validation Loss: 0.331636
Epoch: 3 	Training Loss: 1.331860 	Validation Loss: 0.331734
Epoch: 4 	Training Loss: 1.331540 	Validation Loss: 0.331817
Epoch: 5 	Training Loss: 1.331185 	Validation Loss: 0.331906
Epoch: 6 	Training Loss: 1.331154 	Validation Loss: 0.331971
Epoch: 7 	Training Loss: 1.330937 	Validation Loss: 0.332047
Epoch: 8 	Training Loss: 1.330832 	Validation Loss: 0.332116
Epoch: 9 	Training Loss: 1.330603 	Validation Loss: 0.332169
Epoch: 10 	Training Loss: 1.330338 	Validation Loss: 0.332240
Epoch: 11 	Training Loss: 1.330459 	Validation Loss: 0.332301
Epoch: 12 	Training Loss: 1.330314 	Validation Loss: 0.332377
Epoch: 13 	Training Loss: 1.330116 	Validation Loss: 0.332446
Epoch: 14 	Training Loss: 1.330099 	Validation Loss: 0.332503
Epoch: 15 	Training Loss: 1.329876 	Validation Loss: 0.332533
Epoch: 16 	Trai

KeyboardInterrupt: ignored

In [0]:
#Load the Model with the Lowest Validation Loss
model.load_state_dict(torch.load('drive/My Drive/CNN without cycling/model_augmented2.pt'))

In [0]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(8))
class_total = list(0. for i in range(8))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
   # for i in range(batch_size):
    for i in range(len(target)):
        label = target.data[i]
        if(pred[i] == label):
           class_correct[label] += 1
        else:
           print(label,pred[i]) 
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(8):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))


tensor(4, device='cuda:0') tensor(3, device='cuda:0')
tensor(0, device='cuda:0') tensor(1, device='cuda:0')
tensor(5, device='cuda:0') tensor(2, device='cuda:0')
tensor(5, device='cuda:0') tensor(7, device='cuda:0')
tensor(5, device='cuda:0') tensor(7, device='cuda:0')
tensor(5, device='cuda:0') tensor(7, device='cuda:0')
tensor(1, device='cuda:0') tensor(0, device='cuda:0')
tensor(7, device='cuda:0') tensor(5, device='cuda:0')
tensor(2, device='cuda:0') tensor(4, device='cuda:0')
tensor(5, device='cuda:0') tensor(3, device='cuda:0')
tensor(3, device='cuda:0') tensor(2, device='cuda:0')
tensor(0, device='cuda:0') tensor(6, device='cuda:0')
tensor(2, device='cuda:0') tensor(5, device='cuda:0')
tensor(7, device='cuda:0') tensor(2, device='cuda:0')
tensor(1, device='cuda:0') tensor(0, device='cuda:0')
tensor(4, device='cuda:0') tensor(3, device='cuda:0')
tensor(2, device='cuda:0') tensor(3, device='cuda:0')
tensor(2, device='cuda:0') tensor(5, device='cuda:0')
tensor(2, device='cuda:0') t